# Generating C++ Kokkos Chemistry Solvers with JAFF

This notebook demonstrates how to use JAFF to generate C++ code that uses the Kokkos-kernels BDF solver for solving chemical reaction networks. We'll use the `react_COthin` network as an example, which contains 37 species.

## Overview

The Kokkos output plugin generates:
- **chemistry_ode.hpp**: Chemistry ODE system class with evaluate_function and evaluate_jacobian methods
- **chemistry_ode.cpp**: Main executable that sets up and runs the BDF solver
- **CMakeLists.txt**: Build configuration for Kokkos and Kokkos-kernels dependencies

The generated code is compatible with the Kokkos-kernels BDF solver for stiff ODEs.

## Step 1: Load the Chemical Network

First, let's load the react_COthin network, which is a carbon-oxygen chemistry network in KROME format.

In [1]:
from jaff.network import Network
from jaff.builder import Builder
import os

# Load the test2 network
network = Network("../networks/test2.dat")

print(f"Network loaded: {network.label}")
print(f"Number of species: {network.get_number_of_species()}")
print(f"Number of reactions: {len(network.reactions)}")

Welcome to JAFF: Just Another Fancy Format!
Loading network from ../networks/test2.dat
Network label = test2


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 465.03it/s]

2.0e-12*tgas**0.33
1.0e-12*exp(-1.222*av)
Variables found: ['av', 'tgas']
Loaded 2 reactions
Lodaded 0 photo-chemistry reactions
All done!
Network loaded: test2
Number of species: 3
Number of reactions: 2


## Step 2: Examine Network Properties

Let's examine some properties of the network to understand what we're working with.

In [2]:
for i, species in enumerate(network.species):
    print(f"  {i}: {species.name} (mass: {species.mass:.2e}, charge: {species.charge})")

print(f"\nTotal species: {len(network.species)}")

  0: e- (mass: 9.11e-28, charge: -1)
  1: CO+ (mass: 4.65e-23, charge: 1)
  2: CO (mass: 4.65e-23, charge: 0)

Total species: 3


In [3]:
# Show a few example reactions
for i, reaction in enumerate(network.reactions[:10]):
    reactants = " + ".join([f"{r.name}" for r in reaction.reactants])
    products = " + ".join([f"{p.name}" for p in reaction.products])
    print(f"  {i+1}: {reactants} -> {products}")
    print(f"      Rate: {reaction.rate}")
    print()

  1: e- + CO+ -> CO
      Rate: 2.0e-12*tgas**0.33

  2: CO -> CO+ + e-
      Rate: 1.0e-12*exp(-1.222*av)



## Step 3: Generate C++ Kokkos Code

Now let's use the Builder to generate C++ code using the `kokkos_ode` template.

In [4]:
# Create builder and generate Kokkos C++ code
builder = Builder(network)
builds_dir = builder.build(template="kokkos_ode")

print("\nC++ Kokkos code generated successfully!")
print(f"Generated files are located at {builds_dir}")

Building network with template: kokkos_ode
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/chemistry_ode.hpp -> /Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.hpp
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/chemistry_ode.cpp -> /Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.cpp
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/CMakeLists.txt -> /Users/benwibking/amrex_codes/chemistry/jaff/examples/CMakeLists.txt
Network built successfully using template 'kokkos_ode'.
Output files are located in: /Users/benwibking/amrex_codes/chemistry/jaff/examples

C++ Kokkos code generated successfully!
Generated files are located at /Users/benwibking/amrex_codes/chemistry/jaff/examples


## Step 4: Examine Generated Files

Let's look at the structure and content of the generated files.

In [5]:
# List generated files
generated_files = [f for f in os.listdir(builds_dir) if f.endswith(('.cpp', '.hpp', '.txt'))]

print("Generated files:")
for file in sorted(generated_files):
    filepath = os.path.join(builds_dir, file)
    size = os.path.getsize(filepath)
    print(f"  {file:20} ({size:,} bytes)")

Generated files:
  CMakeLists.txt       (2,371 bytes)
  chemistry_ode.cpp    (4,233 bytes)
  chemistry_ode.hpp    (4,252 bytes)


## Step 5: Understanding the Generated Code Structure

Let's examine key components of the generated chemistry solver.

In [6]:
# Show the ChemistryODE class definition
with open(builds_dir + "/chemistry_ode.hpp", 'r') as f:
    lines = f.readlines()

# Find the class definition
class_start = None
for i, line in enumerate(lines):
    if "struct ChemistryODE" in line:
        class_start = i
        break

if class_start:
    print("ChemistryODE class structure:")
    print("=" * 50)
    # Show class definition and methods
    brace_count = 0
    for i in range(class_start, len(lines)):
        line = lines[i].rstrip()
        print(f"{i+1:2d}: {line}")
        
        if '{' in line:
            brace_count += line.count('{')
        if '}' in line:
            brace_count -= line.count('}')
            if brace_count == 0:
                break

ChemistryODE class structure:
14: struct ChemistryODE {
15:     // Number of species in the chemical network
16:     // PREPROCESS_NUM_SPECIES
17: 
18:     static constexpr int neqs = 3;
19: 
20:     // PREPROCESS_END
21: 
22:     using state_type = std::array<integrators::Real, neqs>;
23:     using rhs_type = std::array<integrators::Real, neqs>;
24:     using jacobian_type = std::array<std::array<integrators::Real, neqs>, neqs>;
25: 
26:     // Species indices and common constants
27:     // PREPROCESS_COMMONS
28: 
29:     static constexpr int idx_e = 0;
30:     static constexpr int idx_coj = 1;
31:     static constexpr int idx_co = 2;
32:     static constexpr int nspecs = 3;
33:     static constexpr int nreactions = 2;
34: 
35:     // Common chemistry variables used in rate expressions
36:     // These should typically be passed as parameters or computed from the state
37:     static constexpr double DEFAULT_TEMPERATURE = 300.0;  // Default gas temperature in K
38:     static constex

## Step 6: Building and Running Instructions

Here's how to build and run the generated C++ code:

* **Configure the build with CMake:**

In [7]:
!   cmake -S . -B build

-- The CXX compiler identification is AppleClang 17.0.0.17000013
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- The Fortran compiler identification is GNU 15.1.0
-- Checking whether Fortran compiler has -isysroot
-- Checking whether Fortran compiler has -isysroot - yes
-- Checking whether Fortran compiler supports OSX deployment target flag
-- Checking whether Fortran compiler supports OSX deployment target flag - yes
-- Detecting Fortran compiler ABI info
-- Detecting Fortran compiler ABI info - done
-- Check for working Fortran compiler: /opt/homebrew/bin/gfortran - skipped
-- Looking for a CUDA compiler
-- Looking for a CUDA compiler - NOTFOUND
-- Configuring done (2.4s)
-- Generating done (0.0s)
-- Build files have been written to: /Users/benwibking/amrex_codes/chemistry/jaff/examples/build


In [8]:
assert(_exit_code == 0)

* **Build with parallel compilation:**

In [9]:
!   cmake --build build -j

[1/2] Building CXX object CMakeFiles/chemistry_ode.dir/chemistry_ode.cpp.o
In file included from /Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.cpp:7:
/Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.hpp:50:22: warning: unused variable 'tdust' [-Wunused-variable]
   50 |         const double tdust = DEFAULT_TEMPERATURE;
      |                      ^~~~~
/Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.hpp:52:22: warning: unused variable 'crate' [-Wunused-variable]
   52 |         const double crate = DEFAULT_CRATE;  // Cosmic ray ionization rate
      |                      ^~~~~
/Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.hpp:76:22: warning: unused variable 'cse3' [-Wunused-variable]
   76 |         const double cse3 = -cse0;
      |                      ^~~~
/Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.hpp:78:22: warning: unused variable 'cse5' [-Wunused-variable]
   78 |       

In [10]:
assert(_exit_code == 0)

* **Run the chemistry solver:**

In [11]:
!   ./build/chemistry_ode

Solving Chemistry ODE System (header-only integrators)
Number of species: 3
Number of reactions: 2
Time interval: [0, 3.15576e+09] seconds
Initial conditions:
  y[0] = 1e-06
  y[1] = 1e-10
  y[2] = 1e-10

Final solution at t = 3155760000 seconds:
  y[0] = 1.000000093e-06
  y[1] = 1.000929311e-10
  y[2] = 9.990706892e-11

Conservation check:
  Initial sum: 1.0002e-06
  Final sum:   1.000200093e-06
  Relative change: 9.291249572e-08


In [12]:
assert(_exit_code == 0)

* **Or, run with custom time interval (1e12 seconds):**

In [17]:
!   ./build/chemistry_ode 1e15

Solving Chemistry ODE System (header-only integrators)
Number of species: 3
Number of reactions: 2
Time interval: [0, 1e+15] seconds
Initial conditions:
  y[0] = 1e-06
  y[1] = 1e-10
  y[2] = 1e-10

Final solution at t = 1e+15 seconds:
  y[0] = 1.000099991e-06
  y[1] = 1.999910825e-10
  y[2] = 8.917475425e-15

Conservation check:
  Initial sum: 1.0002e-06
  Final sum:   1.000299991e-06
  Relative change: 9.997108829e-05


In [14]:
assert(_exit_code == 0)